In [24]:
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
import duckdb
from dash import html, dcc, Output, Input, jupyter_dash, callback, ctx, Dash
import dash_bootstrap_components as dbc
jupyter_dash.default_mode = "external"

In [25]:
con = duckdb.connect()

df = con.query(
    '''
    select * from read_csv("/home/sohail/MyDashApp/DataFolder/mushroom_cleaned.csv")
    '''
).pl()
df = df.rename({col:col.replace("-"," ") for col in df.columns}).select(pl.all().shrink_dtype())
df.head()

cap diameter,cap shape,gill attachment,gill color,stem height,stem width,stem color,season,class
i16,i8,i8,i8,f32,i16,i8,f32,i8
1372,2,2,10,3.807467,1545,11,1.804273,1
1461,2,2,10,3.807467,1557,11,1.804273,1
1371,2,2,10,3.612496,1566,11,1.804273,1
1261,6,2,10,3.787572,1566,11,1.804273,1
1305,6,2,10,3.711971,1464,11,0.943195,1


In [26]:
cat_cols = sorted(cs.expand_selector(df,(cs.integer() | cs.boolean() | cs.string() | cs.categorical())))
cat_cols

['cap diameter',
 'cap shape',
 'class',
 'gill attachment',
 'gill color',
 'stem color',
 'stem width']

In [27]:
cont_cols = sorted(cs.expand_selector(df,cs.float()))
cont_cols

['season', 'stem height']

In [28]:
mushroom_layout = html.Div(
    [
        html.Div(
            [
                html.Ul(
                    html.Li(
                        html.Span(
                            [
                                html.H1("MUSHROOM INDEX", className="subpage-dropdown-button-title"),
                                html.Img(src="assets/mushroom_index_icon.jpg", className="subpage-dropdown-button-icon")
                            ]
                        )
                    ),
                    className="subpage-dropdown-button"
                ),
                html.Div(html.Span(className="arrow"), className="arrow-space"),
                html.Ul(
                    [   
                        html.Li(
                            html.Span(
                                [
                                    html.H1(f"{cat_cols[0].upper()}"),
                                ]
                            )
                        ),
                        html.Li(
                            html.Span(
                                [
                                    html.H1(f"{cat_cols[1].upper()}"),
                                ]
                            )
                        ),
                        html.Li(
                            html.Span(
                                [
                                    html.H1(f"{cat_cols[2].upper()}"),
                                ]
                            )
                        ),
                        html.Li(
                            html.Span(
                                [
                                    html.H1(f"{cat_cols[3].upper()}"),
                                ]
                            )
                        ),
                        html.Li(
                            html.Span(
                                [
                                    html.H1(f"{cat_cols[4].upper()}"),
                                ]
                            )
                        ),
                        html.Li(
                            html.Span(
                                [
                                    html.H1(f"{cat_cols[5].upper()}"),
                                ]
                            )
                        ),
                        html.Li(
                            html.Span(
                                [
                                    html.H1(f"{cat_cols[6].upper()}"),
                                ]
                            )
                        ),
                        html.Li(
                            html.Span(
                                [
                                    html.H1(f"{cont_cols[0].upper()}"),
                                ]
                            )
                        ),
                        html.Li(
                            html.Span(
                                [
                                    html.H1(f"{cont_cols[1].upper()}"),
                                ]
                            )
                        ),
                    ],
                    className="subpage-dropdown-box"
                )
            ],
            className="subpage-total-dropdown",
        ),
        html.Div(id="subpage-page-content")
    ]
)


covid_layout = html.Div(
    [
        dcc.Dropdown(
            options=[
                dict(label="covid FIRST PAGE",value="covid_first_page"),
                dict(label="covid SECOND PAGE",value="covid_second_page"),
                dict(label="covid THIRD PAGE",value="covid_third_page"),
            ],
            optionHeight=100,
            maxHeight=600,
            id="covid-dropdown",
            searchable=False
        ),
        html.Div(id="covid-page-content")
    ]
)

kfc_layout = html.Div(
    [
        dcc.Dropdown(
            options=[
                dict(label="kfc FIRST PAGE",value="kfc_first_page"),
                dict(label="kfc SECOND PAGE",value="kfc_second_page"),
                dict(label="kfc THIRD PAGE",value="kfc_third_page"),
            ],
            optionHeight=100,
            maxHeight=600,
            id="kfc-dropdown",
            searchable=False
        ),
        html.Div(id="kfc-page-content")
    ]
)

mushroom_first_page = html.H1("This is mushroom first page")
mushroom_second_page = html.H1("This is mushroom second page")
mushroom_third_page = html.H1("This is mushroom third page")


covid_first_page = html.H1("This is covid first page")
covid_second_page = html.H1("This is covid second page")
covid_third_page = html.H1("This is covid third page")


kfc_first_page = html.H1("This is kfc first page")
kfc_second_page = html.H1("This is kfc second page")
kfc_third_page = html.H1("This is kfc third page")

In [29]:
app = Dash(assets_folder="/home/sohail/MyDashApp/assets",suppress_callback_exceptions=True)

app.layout = dcc.Loading(
    [
        html.Div(
            [
                html.Ul(
                    html.Li(
                        html.Span(
                            [
                                html.H1("DATASET MENU",className="dropdown-button-title"),
                                html.Img(src="assets/datascience_icon.png",className="dropdown-button-icon")
                            ]
                        )
                    ),
                    className="dropdown-button",
                ),
                html.Div(html.Span(className="arrow"),className="arrow-space"),
                html.Ul(
                    [
                        html.Li(
                            html.Span(
                                [
                                    html.Img(src="assets/index_icon.jpg"),
                                    html.H1("INDEX")
                                ],
                                id="index"
                            )
                        ),
                        html.Li(
                            html.Span(
                                [
                                    html.Img(src="assets/mushroom_icon.jpg"),
                                    html.H1("MUSHROOM")
                                ],
                                id="mushroom"
                            )
                        ),
                        html.Li(
                            html.Span(
                                [
                                    html.Img(src="assets/covid.png"),
                                    html.H1("COVID")
                                ],
                                id="covid"
                            )
                        ),
                        html.Li(
                            html.Span(
                                [
                                    html.Img(src="assets/kfc_icon.png"),
                                    html.H1("KFC STOCK")
                                ],
                                id="kfc_stock"
                            )
                        )
                    ],
                    className="dropdown-box",
                )
            ],
            className="total-dropdown"
        ),
        html.Div(id="main-page-content",className="main-page-class")
    ]
)

index_layout = html.H1("THis is index page")

@callback(
    Output("mushroom-page-content","children"),
    Input("mushroom-dropdown","value")
)
def display_mushroom_page(value):
    if value == "mushroom_first_page":
        return mushroom_first_page
    elif value == "mushroom_second_page":
        return mushroom_second_page
    elif value == "mushroom_third_page":
        return mushroom_third_page

@callback(
    Output("covid-page-content","children"),
    Input("covid-dropdown","value")
)
def display_covid_page(value):
    if value == "covid_first_page":
        return covid_first_page
    elif value == "covid_second_page":
        return covid_second_page
    elif value == "covid_third_page":
        return covid_third_page

@callback(
    Output("kfc-page-content","children"),
    Input("kfc-dropdown","value")
)
def display_kfc_page(value):
    if value == "kfc_first_page":
        return kfc_first_page
    elif value == "kfc_second_page":
        return kfc_second_page
    elif value == "kfc_third_page":
        return kfc_third_page

@callback(
    Output("main-page-content","children"),
    [
        Input("index","n_clicks"),
        Input("mushroom","n_clicks"),
        Input("covid","n_clicks"),
        Input("kfc_stock","n_clicks")
    ]
)
def display_page(*args):
    if ctx.triggered_id == "mushroom":
        return mushroom_layout
    elif ctx.triggered_id == "covid":
        return covid_layout
    elif ctx.triggered_id == "kfc_stock":
        return kfc_layout
    elif ctx.triggered_id == "index":
        return index_layout
    else:
        return mushroom_layout
    


app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
